In [1]:
import numpy as np
import pandas as pd
import datetime
import csv

In [2]:
case_level_full_data = {}

In [3]:
full_charge_data = pd.read_excel('../data/steps/Step5Output_joined.xlsx')
full_charge_data.head()

,arresting_pd,case_id,charge_filing_date,charge_id,charge_number,disposition,disposition_date,dob,first_name,judge,last_name,offense_date,offense_description,sentence,sentence_conditions,sentence_date,sentence_terms,unique_identifier,sup_case,dc_case
0,MIDDLETOWN POLICE DEPARTMENT,7422794,2001-08-04,3909041,1,NaN,NaT,1965-10-27,SCOTT,NaN,DRISCOLL,2001-08-03,Larceny < 500 Person 65+\n,NaN,NaN,NaT,,4232652,NaN,21-2001-01905
1,TIVERTON POLICE DEPARTMENT,7423292,2001-09-25,3909791,1,Plea of Nolo Contendere,2001-10-18,1971-07-04,NED,"Pirraglia (Retired), Robert K.",FISHER,2001-09-24,Larceny < 500 Person 65+\n,Criminal Sentence,Probation 1Y Active: 10/18/2001; Court Cost...,2001-10-18,,4234254,NaN,21-2001-02507
2,Newport Police Department,7424561,2002-03-27,3911694,2,Dismissed 48A,2003-02-25,1983-04-28,JOSEPH,"Pirraglia (Retired), Robert K.",FERREIRA,2002-03-27,FAILURE TO APPEAR FOR SUMMONS,NaN,NaN,NaT,,4650281,NaN,21-2002-00802
3,Newport Police Department,7424561,2002-03-27,3911693,1,Dismissed 48A,2003-02-25,1983-04-28,JOSEPH,"Pirraglia (Retired), Robert K.",FERREIRA,2002-02-28,Larceny < 500 Person 65+\n,NaN,NaN,NaT,,4650281,NaN,21-2002-00802
4,Newport Police Department,7425369,2002-07-24,3913116,1,Plea of Nolo Contendere,2002-07-24,1976-03-25,DION,"Pirraglia (Retired), Robert K.",ARDREY,2002-07-11,Larceny < 500 Person 65+\n,Criminal Sentence,Credit for Time Served 10H Active: 07/24/20...,2002-07-24,,4651070,NaN,21-2002-01767


In [4]:
for row in full_charge_data.iterrows():
    key = (row[1]['dc_case'], row[1]['sup_case'])
    if key not in case_level_full_data:
        case_level_full_data[key] = [row[1]]
    else:
        case_level_full_data[key].append(row[1])    

In [5]:
def is_elder(charge_record):
    elder_keywords = ('65', '60', 'elder')
    for kw in elder_keywords:
        if kw in charge_record['offense_description'].lower():
            return True
    return False

In [6]:
def has_conviction(charge_record):
    disposition = charge_record['disposition']
    conviction_dispositions = (
        'nolo contendere plea to amended charge',
        'plea of nolo contendere',
        'guilty plea as charged',
        'found guilty by jury',
        'found guilty by judge',
        'guilty plea to amended charge',
        'alford plea entered'
    )
    for conviction_disposition in conviction_dispositions:
        if pd.isnull(disposition):
            return False
        if conviction_disposition in disposition.lower():
            return True
    return False

In [7]:
def was_dismissed(charge_record):
    disposition = charge_record['disposition']
    dismissal_dispositions = (
        'dismissed 48a',
        'dismissed 48b',
        'dismissed by judge',
        'dismissal'
    )
    for dismissal_disposition in dismissal_dispositions:
        if pd.isnull(disposition):
            return False
        if dismissal_disposition in disposition.lower():
            return True
    return False

In [8]:
def is_pending(charge_record):
    disposition = charge_record['disposition']
    pending_dispositions = (
        'information signed',
        'information signed/amended charge',
        'information/indictment waived',
        'charge information signed',
        'true bill signed',
        'charge information/indictment waived',
        'true bill signed/amended charge'
    )
    for pending_disposition in pending_dispositions:
        if pd.isnull(disposition):
            return False
        if pending_disposition in disposition.lower():
            return True
    return False

In [9]:
def is_missing(charge_record):
    disposition = charge_record['disposition']
    missing_dispositions = ('nan')
    for missing_disposition in missing_dispositions:
        if pd.isnull(disposition) or missing_disposition in disposition.lower():
            return True
    return False

In [19]:
def get_jailtime(charge_record):
    sentences = charge_record['sentence_conditions']
    if pd.isnull(sentences):
        return 0
    if 'term to serve' in sentences.lower():
        sentence_array = sentences.split('   ')
        for sentence in sentence_array:
            if 'term to serve' in sentence.lower():
                string_term_to_serve = sentence.split(' ')[-1]
                if string_term_to_serve[-1].lower() == 'y':
                    return 365*int(string_term_to_serve[:1])
                elif string_term_to_serve[-1].lower() == 'm':
                    return 12*int(string_term_to_serve[:1])
                elif string_term_to_serve[-1].lower() == 'd':
                    return int(string_term_to_serve[:1])
                else:
                    raise ValueError
    return 0

In [11]:
def has_sentence(charge_record):
    if charge_record['sentence']:
        return True
    return False

In [27]:
count_charged = 0
count_convicted = 0
count_sentenced = 0
count_pending = 0
count_jailed = 0
count_dismissed = 0
count_missing = 0
count_mixed = 0
jailtimes = []
pending_case_years = [0]*18
for key in case_level_full_data:
    charge_records = case_level_full_data[key]
    elder_records = [charge_record for charge_record in charge_records if is_elder(charge_record)]
    if not elder_records:
        raise ValueError(charge_records)
    count_charged += 1
    if any(has_conviction(charge) for charge in elder_records):
        count_convicted += 1
        if any(has_sentence(charge) for charge in elder_records):
            count_sentenced += 1
            for charge in elder_records:
                jailtime_days = get_jailtime(charge)
                if jailtime_days:
                    jailtimes.append(jailtime_days)
                    count_jailed += 1
                    continue
    elif all(was_dismissed(charge) for charge in elder_records):
        count_dismissed += 1
    elif all(is_pending(charge) for charge in elder_records):
        count_pending += 1
    elif all(is_missing(charge) for charge in elder_records):
        count_missing += 1
    else:
        count_mixed += 1

jailtimes.sort()
print(jailtimes)

[1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 6, 6, 6, 6, 6, 6, 7, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 24, 24, 24, 24, 36, 36, 36, 36, 36, 36, 48, 48, 48, 48, 48, 48, 48, 48, 60, 60, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 84, 108, 108, 108, 108, 108, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 730, 730, 730, 730, 730, 730, 730, 730, 730, 730, 730, 730, 730, 730, 730, 1095, 1095, 1095, 1460, 1460, 1825, 1825, 1825, 2190, 2190, 2190, 2190, 2920, 2920, 3285]


In [28]:
print('{} people charged '.format(count_charged))
print('{} of those were convicted'.format(count_convicted))
print('    {} of those were sentenced'.format(count_sentenced))
print('    {} of those were jailed'.format(count_jailed))
print('{} of the total {} had every charge against them dismissed'.format(count_dismissed, count_charged))
print('{} of the total {} have every charge against them pending'.format(count_pending, count_charged))
print('{} of the total {} have a mix of dismissed/pending/convicted charges'.format(count_mixed, count_charged))
print('{} of the total {} are missing dispositions in our data'.format(count_missing, count_charged))
print('{} is the sum of all of those'.format(count_convicted + count_dismissed + count_missing + count_pending))
print('The everage jailtime was {} days, although {}/{} jailed cases were for a year or less'.format(
    sum(jailtimes)/len(jailtimes), len([k for k in jailtimes if int(k) <= 365]) ,count_jailed
))

1061 people charged 
509 of those were convicted
    509 of those were sentenced
    140 of those were jailed
259 of the total 1061 had every charge against them dismissed
151 of the total 1061 have every charge against them pending
0 of the total 1061 have a mix of dismissed/pending/convicted charges
142 of the total 1061 are missing dispositions in our data
1061 is the sum of all of those
The everage jailtime was 366.0785714285714 days, although 110/140 jailed cases were for a year or less
